## Mills CSI MIST comparison

Comparing our metabolite predictions to those from CSI FingerID

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import random

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs


from mist.utils.plot_utils import *
from mist import utils

set_style()

/home/samlg/.conda/envs/ms-gen/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_ikey(smiles):
    return Chem.MolToInchiKey(Chem.MolFromSmiles(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=False)))

## Load data

In [3]:
mgf_file_500 = Path("../data/raw/mills/Mills_mzxml/mgf_export_sirius_filtered_500.mgf")
mgf_file_full = Path("../data/raw/mills/Mills_mzxml/mgf_export_sirius.mgf")

full_mgf_parsed = utils.parse_spectra_mgf(mgf_file_full)
mgf_500_parsed = utils.parse_spectra_mgf(mgf_file_500)

8662it [00:01, 7631.11it/s] 
6438it [00:00, 9190.01it/s] 


In [4]:
full_names = set([f"mills_{i[0]['FEATURE_ID']}" for i in full_mgf_parsed])
filtered_names = set([f"mills_{i[0]['FEATURE_ID']}" for i in mgf_500_parsed])

# print lens
print(f"Full: {len(full_names)}")
print(f"500: {len(filtered_names)}")

Full: 1990
500: 1515


In [5]:
# Need to do a small set of analyses on 4 possible identifications:
# Us with CSI Formulae
# Us with Formula IDS + PubChem
# Us with CSI 

input_folder = Path("../data/raw/mills/")
out_res = Path("../results/2023_05_10_prospective_reanalysis_forms/")
mist_res_dir = Path("../data/raw/mills/model_predictions/")
mist_csi_res_dir = Path("../results/2023_05_10_prospective_reanalysis_forms/mills/")

# Our results files
csi_pubchem_res_dir = Path("../results/2023_05_08_sirius_check/sirius5s_out_summary/")
csi_hmdb_res_dir = Path("../data/paired_spectra/mills/sirius_outputs_structure_summary")

csi_hmdb_res = csi_hmdb_res_dir / f"compound_identifications.tsv"
csi_hmdb_res_formula = csi_hmdb_res_dir  / f"formula_identifications.tsv"

csi_pubchem_res = csi_pubchem_res_dir / f"compound_identifications.tsv"
csi_pubchem_res_formula = csi_pubchem_res_dir  / f"formula_identifications.tsv"

# Get smiles
mist_hmdb_smiles_out = mist_res_dir / "inthmdb/smiles_outputs.tsv"
mist_pubchem_smiles_out = mist_res_dir / "intpubchem/smiles_outputs.tsv"

mist_csi_hmdb_smiles_out = mist_csi_res_dir / "inthmdb/smiles_outputs.tsv"
mist_csi_pubchem_smiles_out = mist_csi_res_dir / "intpubchem/smiles_outputs.tsv"


save_dir = out_res / "csi_compare"
save_dir.mkdir(exist_ok=True)

In [6]:
mist_hmdb_df = pd.read_csv(mist_hmdb_smiles_out, sep="\t")
mist_pubchem_df = pd.read_csv(mist_pubchem_smiles_out, sep="\t")

mist_csi_hmdb_df = pd.read_csv(mist_csi_hmdb_smiles_out, sep="\t")
mist_csi_pubchem_df = pd.read_csv(mist_csi_pubchem_smiles_out, sep="\t")

# Resort so rank == 0 for all of them
mist_hmdb_df = mist_hmdb_df[mist_hmdb_df["rank"] ==0]
mist_pubchem_df = mist_pubchem_df[mist_pubchem_df["rank"] ==0]

mist_csi_hmdb_df = mist_csi_hmdb_df[mist_csi_hmdb_df["rank"] ==0]
mist_csi_pubchem_df = mist_csi_pubchem_df[mist_csi_pubchem_df["rank"] ==0]


# Add ikey column to all of these
mist_hmdb_df["ikey"] = mist_hmdb_df["smi"].apply(to_ikey)
mist_pubchem_df["ikey"] = mist_pubchem_df["smi"].apply(to_ikey)
mist_csi_hmdb_df['ikey']    = mist_csi_hmdb_df["smi"].apply(to_ikey)
mist_csi_pubchem_df['ikey'] = mist_csi_pubchem_df["smi"].apply(to_ikey)

# Add formula column to all
mist_hmdb_df["formula"] = mist_hmdb_df["smi"].apply(utils.form_from_smi)
mist_pubchem_df["formula"] = mist_pubchem_df["smi"].apply(utils.form_from_smi)
mist_csi_hmdb_df['formula']    = mist_csi_hmdb_df["smi"].apply(utils.form_from_smi)
mist_csi_pubchem_df['formula'] = mist_csi_pubchem_df["smi"].apply(utils.form_from_smi)

RDKit WARNING: [18:11:04] WARNING: not removing hydrogen atom without neighbors
[18:11:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:04] WARNING: not removing hydrogen atom without neighbors
[18:11:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: not removing hydrogen atom without neighbors
[18:11:05] WARNING: not removing hydrogen atom without neighbors
[18:11:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: not removing hydrogen atom without neighbors
[18:11:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:11:05] WARNING: 

In [7]:
# Create form and inchikey dicts for all
mist_hmdb_form_dict = dict(zip(mist_hmdb_df["name"], mist_hmdb_df["formula"]))
mist_hmdb_ikey_dict = dict(zip(mist_hmdb_df["name"], mist_hmdb_df["ikey"]))
mist_hmdb_smiles_dict = dict(zip(mist_hmdb_df["name"], mist_hmdb_df["smi"]))

mist_pubchem_form_dict = dict(zip(mist_pubchem_df["name"], mist_pubchem_df["formula"]))
mist_pubchem_ikey_dict = dict(zip(mist_pubchem_df["name"], mist_pubchem_df["ikey"]))

mist_csi_hmdb_form_dict = dict(zip(mist_csi_hmdb_df["name"], mist_csi_hmdb_df["formula"]))
mist_csi_hmdb_ikey_dict = dict(zip(mist_csi_hmdb_df["name"], mist_csi_hmdb_df["ikey"]))
mist_csi_hmdb_smiles_dict = dict(zip(mist_csi_hmdb_df["name"], mist_csi_hmdb_df["smi"]))

mist_csi_pubchem_form_dict = dict(zip(mist_csi_pubchem_df["name"], mist_csi_pubchem_df["formula"]))
mist_csi_pubchem_ikey_dict = dict(zip(mist_csi_pubchem_df["name"], mist_csi_pubchem_df["ikey"]))
mist_csi_pubchem_smiles_dict = dict(zip(mist_csi_pubchem_df["name"], mist_csi_pubchem_df["smi"]))

In [8]:
csi_hmdb_res = csi_hmdb_res_dir / f"compound_identifications.tsv"
csi_hmdb_res_formula = csi_hmdb_res_dir  / f"formula_identifications.tsv"

csi_pubchem_res = csi_pubchem_res_dir / f"compound_identifications.tsv"
csi_pubchem_res_formula = csi_pubchem_res_dir  / f"formula_identifications.tsv"

In [9]:
id_rename = lambda x : f"mills_{x.rsplit('_')[-1]}"
#  read csv so that the first column, without a header, is the index and the header is off by 1 shifted
csi_df = pd.read_csv(csi_hmdb_res, sep="\t")

# 34 rows have non top formula ranked 
#csi_df[csi_df['formulaRank'].values >= 2].shape
csi_id_to_smiles = dict(zip(csi_df['id'].values, csi_df['smiles'].values))
# extract and convert  33_mgf_export_sirius_filtered_500_46 format ids to mills_46
csi_smi_dict = {id_rename(k) : v for k, v in csi_id_to_smiles.items()} 

csi_form_dict = {k: utils.form_from_smi(v) for k, v in csi_smi_dict.items()}
csi_form_dict = dict(zip(csi_df['id'].values, csi_df['molecularFormula'].values))
csi_form_dict = {id_rename(k) : v for k, v in csi_form_dict.items()}

csi_inchi_dict = {k: to_ikey(v) for k, v in csi_smi_dict.items()}

csi_df_forms = pd.read_csv(csi_hmdb_res_formula, sep="\t")
csi_form_ids_dict = dict(zip(csi_df_forms['id'].values, csi_df_forms['molecularFormula'].values))
csi_form_ids_dict = {id_rename(k) : v for k, v in csi_form_ids_dict.items()}

# Repeat for pubchem res
csi_pubchem_df = pd.read_csv(csi_pubchem_res, sep="\t")
csi_pubchem_id_to_smiles = dict(zip(csi_pubchem_df['id'].values, csi_pubchem_df['smiles'].values))
csi_pubchem_smi_dict = {id_rename(k) : v for k, v in csi_pubchem_id_to_smiles.items()}

csi_pubchem_form_dict = dict(zip(csi_pubchem_df['id'].values, csi_pubchem_df['molecularFormula'].values)) 
csi_pubchem_form_dict = {id_rename(k) : v for k, v in csi_pubchem_form_dict.items()}
#csi_pubchem_form_dict = {k: utils.form_from_smi(v) for k, v in csi_pubchem_smi_dict.items()}

csi_pubchem_inchi_dict = {k: to_ikey(v) for k, v in csi_pubchem_smi_dict.items()}
csi_pubchem_df_forms = pd.read_csv(csi_pubchem_res_formula, sep="\t")
csi_pubchem_form_ids_dict = dict(zip(csi_pubchem_df_forms['id'].values, csi_pubchem_df_forms['molecularFormula'].values))
csi_pubchem_form_ids_dict = {id_rename(k) : v for k, v in csi_pubchem_form_ids_dict.items()}

In [10]:
len(csi_inchi_dict)
print(csi_form_dict['mills_46'], mist_hmdb_form_dict['mills_46'])
print(csi_form_dict['mills_44'], mist_hmdb_form_dict['mills_44'])
print(csi_form_dict['mills_420'], mist_hmdb_form_dict['mills_420'])
print(csi_form_dict['mills_1458'], mist_hmdb_form_dict['mills_1458'])

C7H15NO3 C7H15NO3
C5H9NO2 C5H9NO2
C9H11NO2 C9H11NO2
C17H19NO3 C17H19NO3


In [11]:
# Overlapping keys
csi_ex = set(csi_form_dict.keys())
csi_pubchem_ex = set(csi_pubchem_form_dict.keys())
mist_ex = set(mist_hmdb_form_dict.keys())
overlap_ex = csi_ex.intersection(mist_ex)

# Print lengths with nice message
print(f"CSI hmdb has {len(csi_ex)} unique examples")
print(f"CSI pubchem has {len(csi_pubchem_ex)} unique examples")
print(f"MIST has {len(mist_ex)} unique examples")
print(f"CSI and MIST have {len(overlap_ex)} overlapping examples formulas for hmdb")

CSI hmdb has 724 unique examples
CSI pubchem has 1134 unique examples
MIST has 705 unique examples
CSI and MIST have 545 overlapping examples formulas for hmdb


In [12]:
## Equivalent keys
equiv_form = [i for i in overlap_ex if csi_form_dict[i] == mist_hmdb_form_dict[i]]
equiv_ikey = [i for i in overlap_ex if csi_inchi_dict[i] == mist_hmdb_ikey_dict[i]]

# Print lengths
print(f"CSI and MIST have {len(equiv_form)} overlapping examples with equivalent formulas")
print(f"CSI and MIST have {len(equiv_ikey)} overlapping examples with equivalent inchikeys")

CSI and MIST have 355 overlapping examples with equivalent formulas
CSI and MIST have 200 overlapping examples with equivalent inchikeys


In [25]:
# Generate random baseline using HMDB
import pickle
input_p = pickle.load(open("../data/raw/hmdb/hmdb_formulae_inchikey.p", "rb"))
# input_p.keys()

In [14]:
# Get all names on which mist and csi agree
random.seed(1)
form_entries =   [mist_hmdb_form_dict[i] for i in equiv_form]

# Randomly sample from hmdb dict {form: [inchikey]}
rand_smi_dict = {i: random.choice(list(sorted(input_p[i])))[0] for i in form_entries}
rand_inchi_dict = {k: to_ikey(v) for k, v in rand_smi_dict.items()}

rand_smi_dict = {i: rand_smi_dict[mist_hmdb_form_dict[i]] for i in equiv_form}
rand_inchi_dict = {i: rand_inchi_dict[mist_hmdb_form_dict[i]] for i in equiv_form}

In [15]:
# Check ikey overlap between random and mist
# Check ikey overlap between random and csi
rand_ikey_overlap_mist = [i for i in equiv_form if rand_inchi_dict[i] == mist_hmdb_ikey_dict[i]]
rand_ikey_overlap_csi = [i for i in equiv_form if rand_inchi_dict[i] == csi_inchi_dict[i]]
print(f"Random and MIST have {len(rand_ikey_overlap_mist)} overlapping examples with equivalent inchikeys")
print(f"Random and CSI have {len(rand_ikey_overlap_csi)} overlapping examples with equivalent inchikeys")

Random and MIST have 132 overlapping examples with equivalent inchikeys
Random and CSI have 117 overlapping examples with equivalent inchikeys


In [16]:
# Choose set of examples where csi and mist agree but random does not
rand_diff = [i for i in equiv_form if mist_hmdb_ikey_dict[i] == csi_inchi_dict[i]]
rand_diff = [i for i in rand_diff if rand_inchi_dict[i] != mist_hmdb_ikey_dict[i]]

In [17]:
# Print 10 smiles from rand diff with their ids
sample_keys = ["mills_761", "mills_708", "mills_1265", "mills_420",
               "mills_450", "mills_338", "mills_428",
               "mills_1288", "mills_36", "mills_1276",]

# for ind, i in enumerate(rand_diff):
#     print(i, rand_smi_dict[i], mist_smi_dict[i])
#     if ind > 10:
#         break

for ind, i in enumerate(sample_keys):
    print(i, csi_inchi_dict[i] == mist_hmdb_ikey_dict[i], mist_hmdb_smiles_dict[i])

mills_761 True CCOC(=S)Nc1ccc(O)cc1
mills_708 True O=[N+]([O-])c1ccc(O)c(O)c1
mills_1265 True CC(CCC(=O)O)C1CCC2C3C(O)CC4=CC(=O)CCC4(C)C3CC(O)C12C
mills_420 True NC(Cc1ccccc1)C(=O)O
mills_450 True NCC1CCC(C(=O)O)CC1
mills_338 True CCC(C)C(N)C(=O)N1CCCC1C(=O)O
mills_428 True Oc1ncnc2nc[nH]c12
mills_1288 True CC(CCC(=O)O)C1CCC2C3C(O)CC4CC(OS(=O)(=O)O)CCC4(C)C3CCC12C
mills_36 True NC(N)=NCCCC(N)C(=O)O
mills_1276 True CC1CCC2C(C)C3C(CC4C5CCC6CC(O)CCC6(C)C5CCC43C)N2C1


In [18]:
# Find examples where mist and csi differe
# Choose set of examples where csi and mist agree but random does not
rand_diff = [i for i in equiv_form if mist_hmdb_ikey_dict[i] != csi_inchi_dict[i]]
sample_keys = rand_diff[:10]
sample_keys = ["mills_143", "mills_1278", "mills_1166", "mills_170", "mills_40"]
for ind, i in enumerate(sample_keys):
    print(i, csi_inchi_dict[i] == mist_hmdb_ikey_dict[i], csi_smi_dict[i], mist_hmdb_smiles_dict[i])

mills_143 False C1=CC(=CN=C1)C(=O)O O=C(O)c1ccncc1
mills_1278 False CC(CCC(=O)O)C1CCC2C1(CCC3C2C(=O)CC4C3(CCC(C4)O)C)C CCCCCCC(C)(C)c1cc(O)c2c(c1)OC(C)(C)C1CCC(=O)CC21
mills_1166 False CC(CCC(=O)O)C1CCC2C1(C(CC3C2C(CC4C3(CCC(C4)O)C)O)O)C CC(CCC(=O)O)C1CCC2C3C(O)CC4CC(O)C(O)CC4(C)C3CCC12C
mills_170 False CC1C(C(C(C(O1)OCC2C(C(C(C(O2)O)O)O)O)O)O)O CC1OC(OC2C(O)OC(CO)C(O)C2O)C(O)C(O)C1O
mills_40 False CC(=O)NCC(CCC(C(=O)O)N)O NC(CCCCNCC(=O)O)C(=O)O


## Show that when we look at the formula identifications alone, diffs go away

In [19]:
# Compute overlap between csi hmdb form ids and mist forms

csi_mist_form_overlap = [k for k in mist_hmdb_form_dict.keys() if csi_form_ids_dict.get(k) == mist_hmdb_form_dict.get(k)]
csi_mist_pubchem_form_overlap = [k for k in mist_pubchem_form_dict.keys() if csi_form_ids_dict.get(k) == mist_pubchem_form_dict.get(k)]

# print len
print(f"CSI and MIST have {len(csi_mist_form_overlap)} overlapping examples with equivalent formulas of total {len(csi_form_ids_dict)} and {len(mist_hmdb_form_dict)} respectively")
print(f"CSI and MIST have {len(csi_mist_pubchem_form_overlap)} overlapping examples with equivalent formulas of total {len(csi_form_ids_dict)} and {len(mist_pubchem_form_dict)} respectively")

CSI and MIST have 595 overlapping examples with equivalent formulas of total 1355 and 705 respectively
CSI and MIST have 882 overlapping examples with equivalent formulas of total 1355 and 1499 respectively


In [20]:
# Look at differences
# Get all names on which mist and csi disagree on chem formula
csi_mist_form_diff = [k for k in mist_hmdb_form_dict.keys() if csi_form_ids_dict.get(k) != mist_hmdb_form_dict.get(k)]
rnd_sample = csi_mist_form_diff[:10]
for i in rnd_sample:
    print(i, csi_form_ids_dict.get(i), mist_pubchem_form_dict[i])

mills_1293 C27H49NO4 C27H45NO2
mills_1590 None C26H50NO7P
mills_644 C15H25NO8 C16H21N5O4
mills_155 C7H13NO C7H16N2O
mills_660 C12H23N4O5 C15H21N5O2
mills_1302 C16H33N7O6S C24H29N5O4
mills_121 C11H19NO6 C15H19NOS
mills_548 C4H9N7O3 C8H13NO5
mills_796 C15H33N3O6 C15H29N3O4
mills_642 C10H9N C10H12N2


Differences between csi pre and post retrieval

In [21]:
csi_self_overlap = [k for k in csi_form_ids_dict if csi_form_ids_dict.get(k) == csi_pubchem_form_dict.get(k)]
print(f"CSI has {len(csi_self_overlap)} overlapping examples with equivalent formulas of total {len(csi_form_ids_dict)} and {len(csi_pubchem_form_dict)} respectively")

CSI has 625 overlapping examples with equivalent formulas of total 1355 and 1134 respectively


## Look at inchikey overlap between new annots with csi

In [24]:
mist_csi_pubchem_form_dict, mist_csi_pubchem_ikey_dict 
mist_csi_pubchem_csi_overlap = [k for k in mist_csi_pubchem_form_dict if mist_csi_pubchem_form_dict.get(k) == csi_pubchem_form_dict.get(k)]
print(f"MIST and CSI have {len(mist_csi_pubchem_csi_overlap)} overlapping examples with equivalent formulas of total {len(mist_csi_pubchem_form_dict)} and {len(csi_pubchem_form_dict)} respectively")
# Check for ikeys
mist_csi_pubchem_ikey_overlap = [k for k in mist_csi_pubchem_ikey_dict if mist_csi_pubchem_ikey_dict.get(k) == csi_pubchem_inchi_dict.get(k)]
print(f"MIST and CSI have {len(mist_csi_pubchem_ikey_overlap)} overlapping examples with equivalent inchikeys of total {len(mist_csi_pubchem_ikey_dict)} and {len(csi_pubchem_inchi_dict)} respectively")

# Where are differences in the formula between 621 and 642??
mist_csi_pubchem_csi_diff = [k for k in mist_csi_pubchem_form_dict if mist_csi_pubchem_form_dict.get(k) != csi_pubchem_form_dict.get(k)]
print(mist_csi_pubchem_csi_diff)
for i in mist_csi_pubchem_csi_diff:
    print(i, mist_csi_pubchem_form_dict[i], csi_pubchem_form_dict[i], mist_csi_pubchem_ikey_dict[i], csi_pubchem_inchi_dict[i])

MIST and CSI have 640 overlapping examples with equivalent formulas of total 642 and 1134 respectively
MIST and CSI have 162 overlapping examples with equivalent inchikeys of total 642 and 1134 respectively
['mills_1533', 'mills_1342']
mills_1533 C13H17N2O2- C13H17N2O2 CTTCKNKDVKJORZ-UHFFFAOYSA-M DYUUGILMVYJEHY-UHFFFAOYSA-N
mills_1342 C27H32N4O5+2 C27H32N4O5 IWYPBYQQJULZSJ-UHFFFAOYSA-P YENLRELPTNVUIB-UHFFFAOYSA-N


In [23]:
# # Check what fraction of overlap ikeys are in train
# ikey_same_set = {mist_csi_pubchem_ikey_dict[i] for i in mist_csi_pubchem_ikey_overlap}
# ref_set = set(pd.read_csv("../data/paired_spectra/csi2022/labels.tsv", sep="\t")['inchikey'].values)
# in_ref_set = ref_set.intersection(ikey_same_set)

# print(f"{len(in_ref_set)} of {len(ikey_same_set)} overlap ikeys are in train set")

# mist_ikey_total = set(mist_csi_pubchem_ikey_dict.values())
# csi_ikey_total = set(csi_pubchem_inchi_dict.values())
# # Check how many are in ref set from these and print
# mist_ikey_in_ref = mist_ikey_total.intersection(ref_set)
# csi_ikey_in_ref = csi_ikey_total.intersection(ref_set)
# print(f"{len(mist_ikey_in_ref)} of {len(mist_ikey_total)} MIST ikeys are in train set")
# print(f"{len(csi_ikey_in_ref)} of {len(csi_ikey_total)} CSI ikeys are in train set")

# # For fraction that doesn't match, what's the avg tanimoto similarity between the top 1 hits?
# mist_csi_ikey_diff = [k for k in mist_csi_pubchem_ikey_dict if mist_csi_pubchem_ikey_dict.get(k) != csi_pubchem_inchi_dict.get(k)]

# sims = []
# mols_list  = []
# equiv_form = []
# for k in mist_csi_ikey_diff: 
#     s1, s2 = mist_csi_pubchem_smiles_dict[k], csi_pubchem_smi_dict[k]
#     f1, f2 = utils.form_from_smi(s1), utils.form_from_smi(s2)
#     equiv_form.append(f1 == f2)
#     mols = [Chem.MolFromSmiles(s) for s in [s1,s2]]
#     fps = [AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048) for m in mols]
#     sim = DataStructs.TanimotoSimilarity(fps[0], fps[1])
#     sims.append(sim)
#     mols_list.append(mols)
# print(np.mean(equiv_form), np.mean(sims))
# # Confirm
# a, b  = np.array(mols_list), np.array(sims)
# ind = np.random.randint(0, len(a))
# # Print sim nd idisplay mol grid
# print(b[ind])
# # Label with "mist", "csi"
# Draw.MolsToGridImage(a[ind], molsPerRow=2, subImgSize=(300,300), legends=["mist", "csi"])
# # Plot distribution of tanimoto similarities with seaborn
# sns.histplot(sims, bins=20, kde=True, stat="density", alpha=0.5, color="blue") 
# plt.title("Top 1 tani sim from mist/csi in pubchem when not matched")
# plt.xlabel("Tani sim")